# SmartLife Concierge Agent – Personal Productivity Chatbot 🧠🤖

**Track:** Concierge Agents – automate daily tasks and improve individual workflows.

This notebook implements a chat-style AI agent that helps with:
- Task & reminder management
- PDF → notes summarization
- Email/text summarization
- Code explanation (C, Java, Python, etc.)
- Study timetable generation
- General Q&A chatting


In [ ]:
# If running in Colab/Kaggle, install dependencies (uncomment if needed)
# !pip install google-generativeai PyPDF2

import os
import datetime
from typing import List, Dict, Optional

import google.generativeai as genai
from PyPDF2 import PdfReader


In [ ]:
# 🔑 Configure Gemini API
GEMINI_API_KEY = "YOUR_API_KEY_HERE"  # <-- REPLACE THIS WITH YOUR REAL KEY

if GEMINI_API_KEY == "YOUR_API_KEY_HERE":
    raise ValueError("Please set GEMINI_API_KEY before running the agent.")

genai.configure(api_key=GEMINI_API_KEY)

MODEL_NAME = "gemini-1.5-flash"

def call_llm(system_prompt: str, user_prompt: str) -> str:
    """Helper to call the Gemini model with a system + user prompt."""
    model = genai.GenerativeModel(MODEL_NAME)
    full_prompt = system_prompt + "\n\nUser:\n" + user_prompt
    response = model.generate_content(full_prompt)
    return response.text.strip()


In [ ]:
def extract_text_from_pdf(file_path: str, max_pages: Optional[int] = None) -> str:
    """Extract text from a PDF file."""
    reader = PdfReader(file_path)
    pages_to_read = len(reader.pages) if max_pages is None else min(max_pages, len(reader.pages))

    text = []
    for i in range(pages_to_read):
        page = reader.pages[i]
        page_text = page.extract_text()
        if page_text:
            text.append(page_text)
    return "\n".join(text)


In [ ]:
class SmartLifeAgent:
    """Chat-style agent with internal tools for productivity."""
    def __init__(self):
        self.tasks: List[Dict] = []
        self.task_id_counter = 1

        self.base_system_prompt = (
            "You are SmartLife Chat Concierge, a helpful personal productivity assistant for a student/professional. "
            "You respond in clear, simple English, using bullet points when helpful. "
            "You have access to tools like task management, PDF summarization, code explanation, and timetable planning."
        )

    # ---------- TASKS / REMINDERS ----------
    def add_task(self, title: str, due_date_str: Optional[str] = None, notes: str = "") -> Dict:
        """Add a new task with optional due date (YYYY-MM-DD)."""
        due_date = None
        if due_date_str:
            try:
                due_date = datetime.datetime.strptime(due_date_str, "%Y-%m-%d").date()
            except ValueError:
                pass

        task = {
            "id": self.task_id_counter,
            "title": title,
            "due_date": due_date,
            "notes": notes,
            "completed": False,
            "created_at": datetime.datetime.now()
        }
        self.tasks.append(task)
        self.task_id_counter += 1
        return task

    def list_tasks(self, show_completed: bool = True) -> List[Dict]:
        """Return task list."""
        if show_completed:
            return self.tasks
        return [t for t in self.tasks if not t["completed"]]

    def complete_task(self, task_id: int) -> bool:
        """Mark a task as completed by ID."""
        for t in self.tasks:
            if t["id"] == task_id:
                t["completed"] = True
                return True
        return False

    def format_tasks(self, tasks: List[Dict]) -> str:
        """Pretty-print tasks."""
        if not tasks:
            return "No tasks found."
        lines = []
        today = datetime.date.today()
        for t in tasks:
            status = "✅" if t["completed"] else "🕒"
            due_str = ""
            if t["due_date"]:
                if t["due_date"] < today:
                    due_str = f"(Overdue: {t['due_date']})"
                elif t["due_date"] == today:
                    due_str = "(Due today)"
                else:
                    due_str = f"(Due: {t['due_date']})"
            notes_str = f" | Notes: {t['notes']}" if t["notes"] else ""
            lines.append(f"{status} [ID {t['id']}] {t['title']} {due_str}{notes_str}")
        return "\n".join(lines)

    # ---------- PDF SUMMARY ----------
    def summarize_pdf(self, file_path: str, max_pages: int = 5) -> str:
        raw_text = extract_text_from_pdf(file_path, max_pages=max_pages)
        if not raw_text.strip():
            return "I couldn't extract any text from the PDF. Check if it's a scanned image-based PDF."

        system_prompt = (
            self.base_system_prompt +
            "\nYou are summarizing content from a PDF for a student. Provide sections: 'Short Summary', 'Key Points', 'Important Terms'."
        )
        user_prompt = f"Summarize the following content:\n\n{raw_text[:8000]}"
        return call_llm(system_prompt, user_prompt)

    # ---------- TEXT SUMMARY ----------
    def summarize_text(self, text: str, style: str = "short") -> str:
        if style not in ("short", "detailed"):
            style = "short"
        system_prompt = self.base_system_prompt
        user_prompt = (
            f"Summarize the following text in a {style} and clear way, highlight action items if any:\n\n{text}"
        )
        return call_llm(system_prompt, user_prompt)

    # ---------- CODE EXPLAINER ----------
    def explain_code(self, code: str, language: str = "C") -> str:
        system_prompt = (
            self.base_system_prompt +
            "\nYou are also a programming tutor. Explain code in a beginner-friendly way with steps and a dry run table where relevant."
        )
        user_prompt = (
            f"Explain the following {language} code. Do NOT change the code, just explain it.\n"
            "1. Give a high-level explanation.\n"
            "2. Explain important lines.\n"
            "3. Give a dry run example.\n"
            "4. Show the final output for a sample input (if applicable).\n\n"
            f"Code:\n{code}"
        )
        return call_llm(system_prompt, user_prompt)

    # ---------- STUDY PLAN ----------
    def create_study_plan(self,
                          subjects: List[str],
                          exam_date_str: str,
                          hours_per_day: int = 3,
                          weak_subjects: Optional[List[str]] = None) -> str:
        weak_subjects = weak_subjects or []
        try:
            exam_date = datetime.datetime.strptime(exam_date_str, "%Y-%m-%d").date()
        except ValueError:
            return "Invalid exam date format. Please use YYYY-MM-DD."

        today = datetime.date.today()
        if exam_date <= today:
            return "Exam date must be in the future."

        days_left = (exam_date - today).days

        system_prompt = self.base_system_prompt
        user_prompt = (
            "Create a day-wise study plan.\n\n"
            f"Subjects: {subjects}\n"
            f"Weak subjects (need more time): {weak_subjects}\n"
            f"Days left until exam: {days_left}\n"
            f"Study hours per day: {hours_per_day}\n\n"
            "Output format:\n"
            "- Overview\n"
            "- Day-wise plan (Day 1, Day 2, ...)\n"
            "- Daily revision suggestions\n"
            "- Final revision strategy for last 2 days."
        )
        return call_llm(system_prompt, user_prompt)

    # ---------- GENERAL CHAT ----------
    def general_chat(self, message: str) -> str:
        return call_llm(self.base_system_prompt, message)


In [ ]:
agent = SmartLifeAgent()

HELP_TEXT = """    Available commands:

/help
    Show this help message.

/chat <your message>
    Normal conversation with the assistant.

/add_task
    Add a new task. You will be asked for title, due date (YYYY-MM-DD, optional), notes (optional).

/list_tasks
    Show all tasks.

/list_open_tasks
    Show only pending tasks.

/complete_task <task_id>
    Mark task as completed.

/summarize_text
    Paste any long text/email to get a summary.

/summarize_pdf
    Summarize a PDF file by providing its path in the environment.

/explain_code
    Paste code to get explanation, dry run, and output.

/study_plan
    Generate a study timetable based on subjects and exam date.

Type 'exit' to quit the chat loop.
"""

def chat_loop():
    print("🤖 SmartLife Chat Concierge is ready!")
    print("Type /help to see commands. Type 'exit' to stop.\n")

    while True:
        user_input = input("You > ").strip()
        if user_input.lower() == "exit":
            print("Agent > Goodbye! 👋")
            break

        if not user_input:
            continue

        if user_input.startswith("/help"):
            print("Agent >")
            print(HELP_TEXT)

        elif user_input.startswith("/chat"):
            content = user_input[len("/chat"):].strip()
            if not content:
                content = input("Enter your message: ")
            response = agent.general_chat(content)
            print("\nAgent >\n", response, "\n")

        elif user_input.startswith("/add_task"):
            title = input("Task title: ")
            due = input("Due date (YYYY-MM-DD or leave blank): ").strip() or None
            notes = input("Notes (optional): ")
            task = agent.add_task(title=title, due_date_str=due, notes=notes)
            print(f"Agent > Task added with ID {task['id']}.\n")

        elif user_input.startswith("/list_tasks"):
            print("Agent > Your tasks:")
            print(agent.format_tasks(agent.list_tasks(show_completed=True)), "\n")

        elif user_input.startswith("/list_open_tasks"):
            print("Agent > Your open tasks:")
            print(agent.format_tasks(agent.list_tasks(show_completed=False)), "\n")

        elif user_input.startswith("/complete_task"):
            parts = user_input.split()
            if len(parts) < 2:
                task_id_str = input("Enter task ID to mark as completed: ")
            else:
                task_id_str = parts[1]
            try:
                task_id = int(task_id_str)
                ok = agent.complete_task(task_id)
                if ok:
                    print(f"Agent > Task {task_id} marked as completed.\n")
                else:
                    print(f"Agent > Task ID {task_id} not found.\n")
            except ValueError:
                print("Agent > Invalid task ID.\n")

        elif user_input.startswith("/summarize_text"):
            print("Paste your text/email below. End with a single line containing only 'END':")
            lines = []
            while True:
                line = input()
                if line.strip() == "END":
                    break
                lines.append(line)
            raw_text = "\n".join(lines)
            style = input("Summary style (short/detailed) [short]: ").strip().lower() or "short"
            summary = agent.summarize_text(raw_text, style=style)
            print("\nAgent > Summary:\n", summary, "\n")

        elif user_input.startswith("/summarize_pdf"):
            file_path = input("Enter PDF file path (uploaded to environment): ").strip()
            try:
                pages = input("Max pages to read [5]: ").strip()
                max_pages = int(pages) if pages else 5
            except ValueError:
                max_pages = 5
            try:
                summary = agent.summarize_pdf(file_path, max_pages=max_pages)
                print("\nAgent > PDF Summary:\n", summary, "\n")
            except FileNotFoundError:
                print("Agent > File not found. Please check the path.\n")

        elif user_input.startswith("/explain_code"):
            lang = input("Language (C/Java/Python/etc.) [C]: ").strip() or "C"
            print("Paste your code below. End with a single line containing only 'END':")
            lines = []
            while True:
                line = input()
                if line.strip() == "END":
                    break
                lines.append(line)
            code = "\n".join(lines)
            explanation = agent.explain_code(code=code, language=lang)
            print("\nAgent > Code Explanation:\n", explanation, "\n")

        elif user_input.startswith("/study_plan"):
            subs_raw = input("Enter subjects separated by comma: ")
            subjects = [s.strip() for s in subs_raw.split(",") if s.strip()]
            weak_raw = input("Enter weak subjects (comma-separated, optional): ").strip()
            weak_subjects = [s.strip() for s in weak_raw.split(",") if s.strip()] if weak_raw else []
            exam_date = input("Exam date (YYYY-MM-DD): ").strip()
            hours_per_day_raw = input("Study hours per day [3]: ").strip()
            try:
                hours_per_day = int(hours_per_day_raw) if hours_per_day_raw else 3
            except ValueError:
                hours_per_day = 3

            plan = agent.create_study_plan(subjects=subjects,
                                           exam_date_str=exam_date,
                                           hours_per_day=hours_per_day,
                                           weak_subjects=weak_subjects)
            print("\nAgent > Study Plan:\n", plan, "\n")

        else:
            print("Agent > Command not recognized. Type /help to see available commands.\n")


In [ ]:
# Optional: Demo usage

# Add a sample task
agent.add_task("Finish AI project notebook", due_date_str="2025-11-20", notes="Include chatbot + tools")
print("Sample tasks:")
print(agent.format_tasks(agent.list_tasks()))

# Demo summary
demo_text = """Dear student,

This is a reminder that your mid-term examination for Data Structures will be held on 
25 November 2025 at 10:00 AM in Room 301. You must carry your college ID card and 
a black or blue ball-point pen. Calculators are not allowed.

Regards,
Exam Cell"""

# Uncomment the next line after setting your API key:
# print(agent.summarize_text(demo_text, style="short"))


In [ ]:
# To start interactive chat in this environment, run:
# chat_loop()
